In [29]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.0'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [30]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-01-09 18:58:44--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar.1’

postgresql-42.2.9.j 100%[===================>] 892.61K  --.-KB/s    in 0.08s   

2022-01-09 18:58:44 (10.7 MB/s) - ‘postgresql-42.2.9.jar.1’ saved [914037/914037]



In [31]:
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("VideoGames_V1_00").getOrCreate()
spark = SparkSession.builder.appName("VideoGames_V1_00").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [32]:
from pyspark import SparkFiles
# Load in data from S3 into a DataFrame
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [33]:
video_games_df = spark.read.option('header', 'true').csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), inferSchema=True, sep='\t')
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [34]:
#Count number of records in each row

video_games_df.count()

1785997

In [35]:
#Drop empty
video_games_df = video_games_df.dropna()
video_games_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   12039526| RTIS3L2M1F5SM|B001CXYMFS|     737716809|Thrustmaster T-Fl...|     Video Games|          5|            0|          0|   N|                Y|an amazing joysti...|Used this for Eli...| 2015-08-31|
|         US|    9636577| R1ZV7R40OLHKD|B00M920ND6|     569686175|Tonsee 6 buttons ...|     Video Games|          5|    

In [36]:
#Customer table 
customer_df = video_games_df.select(["customer_id"])
customer_df.show()

+-----------+
|customer_id|
+-----------+
|   12039526|
|    9636577|
|    2331478|
|   52495923|
|   14533949|
|    2377552|
|   17521011|
|   19676307|
|     224068|
|   48467989|
|     106569|
|   48269642|
|   52738710|
|   10556786|
|    2963837|
|   23092109|
|   23091728|
|   10712640|
|   17455376|
|   14754850|
+-----------+
only showing top 20 rows



In [37]:
customer_df = customer_df.groupBy("customer_id").count()
customer_df.orderBy("customer_id").select(["customer_id", "count"])
customer_df.show()

+-----------+-----+
|customer_id|count|
+-----------+-----+
|   48670265|    1|
|   49103216|    2|
|    1131200|    1|
|   43076447|    2|
|   46261368|    1|
|    4883305|    5|
|   41192649|    1|
|   40985731|    7|
|   10437900|    2|
|   22245671|    1|
|    2574873|    1|
|    4696154|    1|
|    5621202|    1|
|    5871933|    2|
|   44089812|    1|
|    2845910|    1|
|    5274369|    1|
|   39069693|    2|
|     137793|    1|
|   31914942|    3|
+-----------+-----+
only showing top 20 rows



In [38]:
customer_df = customer_df.withColumnRenamed("count", "customer_count")
customer_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   48670265|             1|
|   49103216|             2|
|    1131200|             1|
|   43076447|             2|
|   46261368|             1|
|    4883305|             5|
|   41192649|             1|
|   40985731|             7|
|   10437900|             2|
|   22245671|             1|
|    2574873|             1|
|    4696154|             1|
|    5621202|             1|
|    5871933|             2|
|   44089812|             1|
|    2845910|             1|
|    5274369|             1|
|   39069693|             2|
|     137793|             1|
|   31914942|             3|
+-----------+--------------+
only showing top 20 rows



In [39]:
customer_df.printSchema()

root
 |-- customer_id: integer (nullable = true)
 |-- customer_count: long (nullable = false)



In [40]:
#Products table
products_df = video_games_df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B001CXYMFS|Thrustmaster T-Fl...|
|B00M920ND6|Tonsee 6 buttons ...|
|B0029CSOD2|Hidden Mysteries:...|
|B00GOOSV98|GelTabz Performan...|
|B00Y074JOM|Zero Suit Samus a...|
|B002UBI6W6|Psyclone Recharge...|
|B008XHCLFO|Protection for yo...|
|B00BRA9R6A|   Nerf 3DS XL Armor|
|B009EPWJLA|One Piece: Pirate...|
|B0000AV7GB|Playstation 2 Dan...|
|B00008KTNN|Metal Arms: Glitc...|
|B000A3IA0Y|72 Pin Connector ...|
|B0053BQN34|uDraw Gametablet ...|
|B002I0HIMI|NBA 2K12(Covers M...|
|B0081EH12M|New Trigger Grips...|
|B005FMLZQQ|Xbox 360 Media Re...|
|B002BSA388|Super Mario Galaxy 2|
|B00BUSLSAC|Nintendo 3DS XL -...|
|B00KWF38AW|Captain Toad:  Tr...|
|B00BRQJYA8|Lego Batman 2: DC...|
+----------+--------------------+
only showing top 20 rows



In [41]:
products_df = products_df.dropDuplicates(["product_id"])

In [42]:
products_df.printSchema()

root
 |-- product_id: string (nullable = true)
 |-- product_title: string (nullable = true)



In [43]:
#Review ID table
review_id_df = video_games_df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
| RTIS3L2M1F5SM|   12039526|B001CXYMFS|     737716809| 2015-08-31|
| R1ZV7R40OLHKD|    9636577|B00M920ND6|     569686175| 2015-08-31|
|R3BH071QLH8QMC|    2331478|B0029CSOD2|      98937668| 2015-08-31|
|R127K9NTSXA2YH|   52495923|B00GOOSV98|      23143350| 2015-08-31|
|R32ZWUXDJPW27Q|   14533949|B00Y074JOM|     821342511| 2015-08-31|
|R3AQQ4YUKJWBA6|    2377552|B002UBI6W6|     328764615| 2015-08-31|
|R2F0POU5K6F73F|   17521011|B008XHCLFO|      24234603| 2015-08-31|
|R3VNR804HYSMR6|   19676307|B00BRA9R6A|     682267517| 2015-08-31|
| R3GZTM72WA2QH|     224068|B009EPWJLA|     435241890| 2015-08-31|
| RNQOY62705W1K|   48467989|B0000AV7GB|     256572651| 2015-08-31|
|R1VTIA3JTYBY02|     106569|B00008KTNN|     384411423| 2015-08-31|
|R29DOU8791QZL8|   48269642|B000A3IA0Y|     472622859| 2015-08

In [44]:
review_id_df = review_id_df.dropDuplicates(["review_id"])

In [58]:
from pyspark.sql.functions import to_date
review_id_df = review_id_df.withColumn("review_date", to_date("review_date", "yyyy-mm-dd"))
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R10003I619LWL0|    4317219|B001REZLY8|     816407837| 2015-01-10|
|R100078OO83YQB|   31850030|B0016HM45K|     670563392| 2008-01-29|
|R10009XH2FY9Q0|   51098385|B00000IOQV|     845394808| 1999-01-28|
|R1000EJULTHQ16|   12211018|B002I0H7K6|      22716581| 2012-01-05|
|R1000GECIM9DZG|   10430321|B006SVUQNQ|     523228689| 2014-01-06|
|R1000ID99V4TKO|    4971033|B00KY1HZ80|     886096191| 2015-01-18|
|R1000U9GM6NPGS|   13626862|B00DUJKIO6|     518264079| 2014-01-23|
|R1000WZPINB01L|   43132564|B0041HROGG|     166509481| 2012-01-07|
|R1000XAVJH7WUV|   25431956|B000087L4G|     947390207| 2005-01-09|
|R1000YD5XB0Q9H|   45490347|B0093FO2TS|     855418767| 2015-01-26|
|R10010Z0JHJ7RN|   43335519|B000FII8KQ|     613059868| 2014-01-27|
|R10018ETU2IIN1|   12155461|B002I0JGDM|     118325617| 2010-01

In [59]:
review_id_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- customer_id: integer (nullable = true)
 |-- product_id: string (nullable = true)
 |-- product_parent: integer (nullable = true)
 |-- review_date: date (nullable = true)



In [46]:
#Vine table
vine_df = video_games_df.select(["review_id", "star_rating", "helpful_votes", "total_votes"])
vine_df.show()

+--------------+-----------+-------------+-----------+
|     review_id|star_rating|helpful_votes|total_votes|
+--------------+-----------+-------------+-----------+
| RTIS3L2M1F5SM|          5|            0|          0|
| R1ZV7R40OLHKD|          5|            0|          0|
|R3BH071QLH8QMC|          1|            0|          1|
|R127K9NTSXA2YH|          3|            0|          0|
|R32ZWUXDJPW27Q|          4|            0|          0|
|R3AQQ4YUKJWBA6|          1|            0|          0|
|R2F0POU5K6F73F|          5|            0|          0|
|R3VNR804HYSMR6|          5|            0|          0|
| R3GZTM72WA2QH|          5|            0|          0|
| RNQOY62705W1K|          4|            0|          0|
|R1VTIA3JTYBY02|          5|            0|          0|
|R29DOU8791QZL8|          1|            0|          0|
|R15DUT1VIJ9RJZ|          2|            0|          0|
|R3IMF2MQ3OU9ZM|          4|            0|          0|
|R23H79DHOZTYAU|          1|            1|          1|
| RIV24EQA

In [47]:
vine_df = vine_df.dropDuplicates(["review_id"])

In [48]:
vine_df.count()

1785886

In [49]:
vine_df.printSchema()

root
 |-- review_id: string (nullable = true)
 |-- star_rating: integer (nullable = true)
 |-- helpful_votes: integer (nullable = true)
 |-- total_votes: integer (nullable = true)



In [50]:
mode = "append"
jdbc_url = "jdbc:postgresql://mypostgresdb.cqxo6vsypirs.us-east-1.rds.amazonaws.com:5432/mypostgresdb"
config = {"user": "root", "password":"password1", "driver": "org.postgresql.Driver"}

In [51]:
customer_df.write.jdbc(url=jdbc_url, table="customers", mode = mode, properties = config)


In [52]:
products_df.write.jdbc(url=jdbc_url, table="products", mode = mode, properties = config)


In [60]:
review_id_df.write.jdbc(url=jdbc_url, table="review_id_table", mode = mode, properties = config)



In [55]:
vine_df.write.jdbc(url=jdbc_url, table="vine_table", mode = mode, properties = config)